In [1]:
import tensorflow as tf

tf.__version__
tf.__path__
tf.test.is_gpu_available()
tf.test.gpu_device_name()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


'/device:GPU:0'

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
# import preprocess
# from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder


from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss

import random
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model

from sqlalchemy import create_engine
from sqlalchemy.ext.compiler import compiles
from sqlalchemy.sql.expression import Insert

import traceback;
import datetime

pd.set_option('display.min_rows', 50)
pd.set_option('display.max_columns', None) #显示所有列
# pd.set_option('display.max_rows', None) #显示所有行

# 加载数据

In [3]:
# ! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip 
# ! wget https://raw.githubusercontent.com/shenweichen/DeepMatch/master/examples/preprocess.py -O preprocess.py
# ! unzip -o ml-1m.zip

In [3]:
data_path = "./"

# 用户ID，性别，年龄，职业，zip（外加：地理位置如国家城市）
unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)

# 用户ID，电影ID，评分，时间
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)

# 电影ID，电影名，类型
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames)

# 将三个数据表拼成一个表
data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:5000]
print(data.shape)
data.head(5)

<ipython-input-3-6db5c05a1769>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
<ipython-input-3-6db5c05a1769>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
<ipython-input-3-6db5c05a1769>:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie

(1000209, 10)


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [24]:
# 用户数量和电影数量
len(data['user_id'].unique()),len(data['movie_id'].unique())

(6040, 3706)

In [8]:
def gen_data_set(data, negsample=1):
    # 按照时间戳进行升序排序
    data.sort_values("timestamp", inplace=True)
    # 获取唯一的电影id
    item_ids = data['movie_id'].unique()

    train_set = []
    test_set = []
    
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        # 获取当前用户看过的电影ID
        pos_list = hist['movie_id'].tolist()
        # 获取当前用户看过的电影的评分
        rating_list = hist['rating'].tolist()
        
        
        if len(pos_list) == 1:
            train_set.append((reviewerID, [pos_list[0]], pos_list[0], 1,1,rating_list[0]))
        # 如果负样本数大于0
        if negsample > 0:
            # 获取未被看过的电影ID
            # 可以将候选集生成的过程看成是一个极端的多分类问题。那么推荐问题就转化成了一个预测分类准确性的问题。
            candidate_set = list(set(item_ids) - set(pos_list))
            # 随机获取负采样列表
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        
        # 当 len(pos_list)>=2时才会有值, 且数字为从1开始的整数, 也就是只有当用户看过两部以上的电影时, 才会被放入训练集
        for i in range(1, len(pos_list)):
            # 获取除去最后一次看过的电影ID序列
            hist = pos_list[:i]
            # 当数组长度不是最后一个电影数据时
            if i != len(pos_list) - 1:
                # 将数据分割为 除去最后一次历史看过的电影ID序列, 最后一次看过的电影ID，[::-1]含义是将数组倒叙排列, 设置正样本值为1
                # hist[::-1]为 用户观看的movie序列特征，根据观看的时间倒排，即最新观看的movieID排在前面
                # len(hist[::-1]) 为 用户观看的movie序列长度特征，连续特征；
                train_set.append((reviewerID, hist[::-1], pos_list[i], 1, len(hist[::-1]),rating_list[i]))
                
                # 使用没有评分记录的数据, 生成少量负样本, 放入训练集合
                for negi in range(negsample):
                    #[::-1]含义是将数组倒叙排列, 设置负样本值为0
                    train_set.append((reviewerID, hist[::-1], neg_list[i*negsample+negi], 0,len(hist[::-1])))
            else:
                # 将最长的那一个序列长度作为测试数据
                test_set.append((reviewerID, hist[::-1], pos_list[i],1,len(hist[::-1]),rating_list[i]))
    # 随机洗牌，打乱顺序
    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))

    return train_set,test_set

def gen_model_input(train_set,user_profile,seq_max_len):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])
    
    # 用0填补缺失值
    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    # 标签 movie_id: 最后一次的点击视频id号
    train_model_input = {"user_id": train_uid, "movie_id": train_iid, "hist_movie_id": train_seq_pad,"hist_len": train_hist_len}
    # 添加用户信息
    for key in ["gender", "age", "occupation", "zip"]:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

# 构建特征列，训练模型，导出embedding

In [9]:
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip", ]
SEQ_LEN = 50
negsample = 1

# 1. 首先对于数据中的特征进行ID化编码，然后使用 `gen_date_set` and `gen_model_input`来生成带有用户历史行为序列的特征数据


features = ['user_id', 'movie_id', 'gender', 'age', 'occupation', 'zip']
feature_max_idx = {}
for feature in features:
    lbe = LabelEncoder()
    # 将每一列数据标签化并+1
    data[feature] = lbe.fit_transform(data[feature]) + 1
    # 获取标签化后每列数据的最大值+1
    feature_max_idx[feature] = data[feature].max() + 1

    
# 构建用户画像
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
# 构建物品画像
item_profile = data[["movie_id"]].drop_duplicates('movie_id')

user_profile.set_index("user_id", inplace=True)

# 用户历史点击文章序列
user_item_list = data.groupby("user_id")['movie_id'].apply(list)

train_set, test_set = gen_data_set(data, negsample)


train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

# 2.计算每个稀疏字段的独特特征并为序列特征生成特征配置

# 配置一下模型定义需要的特征列，主要是特征名和embedding词表的大小
embedding_dim = 32

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat("gender", feature_max_idx['gender'], 16),
                        SparseFeat("age", feature_max_idx['age'], 16),
                        SparseFeat("occupation", feature_max_idx['occupation'], 16),
                        SparseFeat("zip", feature_max_idx['zip'], 16),
                        VarLenSparseFeat(SparseFeat('hist_movie_id', feature_max_idx['movie_id'], embedding_dim,
                                                    embedding_name="movie_id"), SEQ_LEN, 'mean', 'hist_len'),
                        ]

item_feature_columns = [SparseFeat('movie_id', feature_max_idx['movie_id'], embedding_dim)]



100%|██████████| 6040/6040 [00:25<00:00, 232.52it/s]


5 6
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
start_time = datetime.datetime.now()
print(f"算法开始执行:{start_time}")

# 3.搭建模型并训练

# 定义一个YoutubeDNN模型，分别传入用户侧特征列表user_feature_columns和物品侧特征列表item_feature_columns。然后配置优化器和损失函数，开始进行训练。

K.set_learning_phase(True)

import tensorflow as tf
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()

model = YoutubeDNN(user_feature_columns, item_feature_columns, num_sampled=100, user_dnn_hidden_units=(128,64, embedding_dim))
# model = MIND(user_feature_columns,item_feature_columns,dynamic_k=False,p=1,k_max=2,num_sampled=100,user_dnn_hidden_units=(128,64, embedding_dim))

# 配置优化器和损失函数
model.compile(optimizer="adam", loss=sampledsoftmaxloss)  # "binary_crossentropy")

history = model.fit(train_model_input, train_label,  # train_label,
                    batch_size=8192, epochs=20, verbose=1, validation_split=0.2, )

# 4. 生成用于测试的用户特征和用于检索的完整项目特征
train_user_model_input = train_model_input####

test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

# 以下两行是deepmatch中的通用使用方法，分别获得用户向量模型和物品向量模型
user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)


# 输入对应的数据拿到对应的向量
user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_dict = {"device_ids": train_user_model_input.get("device_id"), "user_embs": user_embs}####

# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)
# item_dict = {"label_id": test_user_model_input.get("label_id"), "item_embs": item_embs}####

print(user_embs.shape)
print(item_embs.shape)

end_time = datetime.datetime.now()
time_cost = end_time - start_time
print(f"算法执行完成:{end_time}\n总耗时: {time_cost}")

算法开始执行:2021-09-27 15:47:52.915670
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Train on 1581006 samples, validate on 395252 samples
Epoch 1/20
1564672/1581006 [============================>.] - ETA: 0s - loss: 3.8729

/opt/miniconda39/envs/tf2/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1581006/1581006 [==============================] - 6s 4us/sample - loss: 3.8737 - val_loss: 3.8670
Epoch 2/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8491 - val_loss: 3.8747
Epoch 3/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8402 - val_loss: 3.8702
Epoch 4/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8368 - val_loss: 3.8295
Epoch 5/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8352 - val_loss: 3.8378
Epoch 6/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8346 - val_loss: 3.8018
Epoch 7/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.7959 - val_loss: 3.7734
Epoch 8/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.8011 - val_loss: 3.7928
Epoch 9/20
1581006/1581006 [==============================] - 4s 3us/sample - loss: 3.7750 - val_loss: 3.7726
Epoch 10/20
1581006/1

# 使用faiss进行ANN查找并评估结果

In [14]:
test_true_label = {line[0]:[line[2]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)#32
# faiss.normalize_L2(item_embs)
index.add(item_embs)#3706*32
# faiss.normalize_L2(user_embs)6040*32


# D:最近邻居的距离，shape (n, k),当没有找到足够多的结果时标签设置为 +Inf 或 -Inf
# I:最近邻居的标签，shape (n, k),当没有找到足够的结果时，标签设置为 -1
D, I = index.search(np.ascontiguousarray(user_embs), 50)
s = []
hit = 0

# enumerate获取用户ID及对应的索引
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['movie_id'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=50)
        s.append(recall_score)
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
# 将多次抽样的计算的结果的平均值当作最终的recall
print("recall：  ", np.mean(s))
print("hit-rate：", hit / len(test_user_model_input['user_id']))

6040it [00:01, 4412.97it/s]


recall：   0.11937086092715232
hit-rate： 0.11937086092715232


In [9]:
def recall_test(test_set,embedding_dim,item_embs,user_embs,test_user_model_input,item_profile):    
    test_true_label = {line[0]:[line[2]] for line in test_set}

    import numpy as np
    import faiss
    from tqdm import tqdm
    from deepmatch.utils import recall_N

    index = faiss.IndexFlatIP(embedding_dim)
    # faiss.normalize_L2(item_embs)
    index.add(item_embs)
    # faiss.normalize_L2(user_embs)


    # D:最近邻居的距离，shape (n, k),当没有找到足够多的结果时标签设置为 +Inf 或 -Inf
    # I:最近邻居的标签，shape (n, k),当没有找到足够的结果时，标签设置为 -1
    D, I = index.search(np.ascontiguousarray(user_embs), 50)
    s = []
    hit = 0

    # enumerate获取用户ID及对应的索引
    for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
        try:
            pred = [item_profile['movie_id'].values[x] for x in I[i]]
            filter_item = None
            recall_score = recall_N(test_true_label[uid], pred, N=50)
            s.append(recall_score)
            if test_true_label[uid] in pred:
                hit += 1
        except:
            print(i)
    print("")
    # 将多次抽样的计算的结果的平均值当作最终的recall
    print("recall：  ", np.mean(s))
    print("hit-rate：", hit / len(test_user_model_input['user_id']))

In [10]:
recall_test(test_set,embedding_dim,item_embs,user_embs,test_user_model_input,item_profile)

6040it [00:01, 4409.32it/s]


recall：   0.10264900662251655
hit-rate： 0.10264900662251655


In [11]:
import pydot_ng as pydot

from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, rankdir="TB", to_file="./imgs/model.png")

plot_model(user_embedding_model, show_shapes=True, show_layer_names=True, rankdir="TB", to_file="./imgs/dnn.png")

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


# 模型保存与加载

In [ ]:
# 要保存/加载权重，您可以像任何其他 keras 模型一样编写代码。
model = YoutubeDNN()
model.save_weights('YoutubeDNN_w.h5')
model.load_weights('YoutubeDNN_w.h5')

In [ ]:
# 保存/加载模型，只是有点不同。

from tensorflow.python.keras.models import  save_model,load_model
model = DeepFM()
save_model(model, 'YoutubeDNN.h5')# save_model, same as before

from deepmatch.layers import custom_objects
model = load_model('YoutubeDNN.h5',custom_objects)# load_model,just add a parameter

# 设置学习率并使用earlystopping

In [ ]:
#您可以在 DeepCTR 中使用任何模型，例如 keras 模型对象。以下是如何设置学习率和提前停止的示例：

import deepmatch
from tensorflow.python.keras.optimizers import Adam,Adagrad
from tensorflow.python.keras.callbacks import EarlyStopping

model = deepmatch.models.FM(user_feature_columns,item_feature_columns)
model.compile(Adagrad(0.01),'binary_crossentropy',metrics=['binary_crossentropy'])

es = EarlyStopping(monitor='val_binary_crossentropy')
history = model.fit(model_input, data[target].values,batch_size=256, epochs=10, verbose=2, validation_split=